In [0]:
# --- PARAMETERS (set by ADF) ---
dbutils.widgets.text("db", "mining")
dbutils.widgets.text("silver_table", "silver_flotation")
dbutils.widgets.text("gold_table", "gold_daily_kpis")
dbutils.widgets.text("silica_threshold", "2.0")
DB = dbutils.widgets.get("db")
SILVER = f"{DB}.{dbutils.widgets.get('silver_table')}"
GOLD = f"{DB}.{dbutils.widgets.get('gold_table')}"
TH = float(dbutils.widgets.get("silica_threshold"))

from pyspark.sql import functions as F

silver = spark.table(SILVER)

daily = (silver
    .withColumn("day", F.to_date("ts"))
    .groupBy("day")
    .agg(
        F.avg("percent_silica_concentrate").alias("avg_silica"),
        F.max("percent_silica_concentrate").alias("max_silica"),
        F.min("percent_silica_concentrate").alias("min_silica"),
        (F.sum(F.when(F.col("percent_silica_concentrate") > TH, 1).otherwise(0))
         / F.count("*") * 100).alias("pct_above_threshold"),
        F.count("*").alias("records")
    )
    .orderBy("day"))

(daily.write
   .format("delta")
   .mode("overwrite")
   .option("overwriteSchema","true")
   .saveAsTable(GOLD))

print(f"✅ Gold built: {GOLD} (threshold={TH})")